# 1. Импорты

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import numpy as np


# 2. Чтение данных

In [ ]:
df = pd.read_csv("/content/meatinfo.csv", sep=';', comment='#')
df

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
...,...,...
17888,"Ягнятина, шея, бк",Ягнятина
17889,Язык ягненка (н.зеландия) Отварные языки ягнят...,Ягнятина
17890,"Ягнятина, язык, зачищ. цена 100 руб.",Ягнятина
17891,"Як, задние части, 1 категория цена 550 руб.",Як


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17893 entries, 0 to 17892
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17893 non-null  object
 1   mtype   17892 non-null  object
dtypes: object(2)
memory usage: 279.7+ KB


In [ ]:
df.describe()

,text,mtype
count,17893,17892
unique,17889,49
top,печень говяжья,Говядина
freq,2,8422


# 3. Отбор классов

In [ ]:
df.mtype.unique()

array(['Баранина', 'Ягнятина', 'Индейка', 'Говядина', 'Свинина', 'Кура',
       'Цыпленок', 'Гусь', 'Буйволятина', 'Оленина', 'Конина', 'Телятина',
       '125р.', 'Кролик', 'Утка', 'Куропатка', 'Парагвай',
       'Говядина, полутуши, 1 категория,  охл., Россия, подвес, В наличии, 10 тонн, 270 руб. кг',
       'Перепел', 'Глухарь', 'Страус', nan, 'Заяц', 'Кенгуру', 'Изюбр',
       'Кабан', '295,00 руб|кг', 'Коза', 'Косуля',
       ' Лопаточная часть (Chuck) буйвол ', 'Лось', 'Марал',
       'Медвежатина', 'Бобер', 'Цесарка', 'Нутрия', 'Feb-20', 'Mar-20',
       '(OFFAL EXP №4407 Аргентина)', 'OFFAL EXP №4407 Аргентина',
       'индейка', 'свиниеа', 'утка', 'цыпленок', 'свинина', 'Рябчик',
       'Тетерев', 'говядина', 'Фазан', 'Як'], dtype=object)

In [ ]:
df.mtype.value_counts()

mtype
Говядина                                                                                   8422
Свинина                                                                                    3050
Кура                                                                                       1571
Индейка                                                                                    1337
Баранина                                                                                   1116
Цыпленок                                                                                    942
Кролик                                                                                      334
Утка                                                                                        195
Оленина                                                                                     193
Конина                                                                                      176
Гусь                              

In [ ]:
col = ["Говядина", "Свинина", "Кура", "Индейка", "Баранина", "Цыпленок"]

In [ ]:
df = df.loc[(df.mtype.isin(col))]
df

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
...,...,...
17816,"Цыпленок, четвертина, задняя",Цыпленок
17817,"Цыпленок, четвертина, задняя цена 100 руб.",Цыпленок
17818,ЦБ Шеи п/ф Свеженка ГОСТ зам пак Шеи куриные П...,Цыпленок
17819,"Цыпленок, шея, без кожи",Цыпленок


# 4. Преобразование данных

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Проверка и очистка данных
df['text'] = df['text'].astype(str)

# Кодирование меток
label_encoder = LabelEncoder()
df['mtype'] = label_encoder.fit_transform(df['mtype'])

# Разделение данных
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['mtype'].tolist(), df['mtype'].tolist(), test_size=0.2, random_state=42
)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16438 entries, 0 to 17820
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16438 non-null  object
 1   mtype   16438 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 385.3+ KB


<ipython-input-9-b85127e53dd8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)
<ipython-input-9-b85127e53dd8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mtype'] = label_encoder.fit_transform(df['mtype'])


In [ ]:
df

,text,mtype
0,12 частей баранина 12 частей баранина,0
1,"Баранина, 12 частей, зам. цена 260 руб.",0
2,"Баранина, 12 частей, зам. цена 315 руб.",0
3,"Баранина, 12 частей, охл.",0
4,"Баранина, 12 частей, охл. цена 220 руб.",0
...,...,...
17816,"Цыпленок, четвертина, задняя",5
17817,"Цыпленок, четвертина, задняя цена 100 руб.",5
17818,ЦБ Шеи п/ф Свеженка ГОСТ зам пак Шеи куриные П...,5
17819,"Цыпленок, шея, без кожи",5


# 5. Создание DataLoader

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

class ProductDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    ds = ProductDataset(
        texts=texts,
        labels=labels,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
    )

BATCH_SIZE = 16
MAX_LEN = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_data_loader = create_data_loader(train_texts, train_labels, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_texts, val_labels, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. Определение и настройка модели

In [ ]:
from transformers import BertForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_encoder.classes_)).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 7. Обучение и валидация модели

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * 3  # Количество эпох

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = torch.nn.CrossEntropyLoss().to(device)


In [ ]:
def train_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    n_examples
):
    model = model.train()

    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)


# 8. Запуск обучения

In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_texts)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val_texts)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

Epoch 1/3
----------
Train loss 0.34548826143041506 accuracy 0.91246332262121
Val   loss 0.1612861159228487 accuracy 0.960324112880693

Epoch 2/3
----------
Train loss 0.14665757621581244 accuracy 0.9558474221042336
Val   loss 0.13463748891600907 accuracy 0.9611623358480023

Epoch 3/3
----------
Train loss 0.12119280201039542 accuracy 0.9608774626240044
Val   loss 0.12644778589934244 accuracy 0.9608829281922324



# 9. Сохранение модели

In [ ]:
model.save_pretrained('bert_product_classification')
tokenizer.save_pretrained('bert_product_classification')

('bert_product_classification/tokenizer_config.json',
 'bert_product_classification/special_tokens_map.json',
 'bert_product_classification/vocab.txt',
 'bert_product_classification/added_tokens.json')

# 10. Загрузка модели и токенизатора

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert_product_classification')
tokenizer = BertTokenizer.from_pretrained('bert_product_classification')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict(text, model, tokenizer, max_len=128):
    model.eval()

    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, prediction = torch.max(outputs.logits, dim=1)

    return prediction.item()

# 11. Пример использования

In [ ]:
ex_text_1 = 'Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.'
ex_text_2 = 'Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:'
ex_text_3 = 'Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.'

texts = [ex_text_1, ex_text_2, ex_text_3]

for i in range(3):
  pred = predict(texts[i], model, tokenizer)
  pred_label = label_encoder.inverse_transform([pred])[0]
  print(f'Текст: {texts[i]}\nПредсказанный класс: {pred_label}\n')

Текст: Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.
Предсказанный класс: Говядина

Текст: Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:
Предсказанный класс: Кура

Текст: Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.
Предсказанный класс: Говядина

